In [1]:
import os

## Set directory
os.chdir('/hpc/group/pbenfeylab/CheWei/CW_data/genesys')

from genesys_train_v1 import *
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
## Conda Env pytorch-gpu on DCC
## pytorch version
print(torch.__version__)
## scanpy
print(sc.__version__) 

1.13.0.post200
1.9.1


In [3]:
## Genes considered/used (shared among samples) 
gene_list = pd.read_csv('./gene_list_1108.csv')

#### Load root atlas data that is going to used for GeneSys training

In [4]:
adata = sc.read_h5ad("/hpc/group/pbenfeylab/CheWei/scRNA-seq/Integrated_Objects/rc.integrated.integrated.h5ad")
meta = pd.read_csv("/hpc/group/pbenfeylab/CheWei/scRNA-seq/Integrated_Objects/Root_Atlas_meta.csv", index_col=0)
adata.obs['celltype.anno'] = meta['celltype.anno']
adata.obs['consensus.time'] = meta['pseudotime']

/hpc/group/pbenfeylab/ch416/miniconda3/envs/pytorch-gpu/lib/python3.8/site-packages/anndata/compat/__init__.py:232: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  warn(


In [5]:
adata

AnnData object with n_obs × n_vars = 110427 × 17513
    obs: 'orig.ident', 'seurat_clusters', 'celltype.anno', 'time.anno', 'consensus.time.group', 'branch.anno', 'Long.ID.P', 'ploidy.ID.P', 'consensus.time'
    var: 'SCT_features', '_index', 'features'
    uns: 'neighbors'
    obsm: 'X_pca', 'X_umap', 'X_umap_2D', 'X_umap_3D', 'X_umap_50'
    varm: 'PCs'
    layers: 'SCT'
    obsp: 'distances'

In [6]:
##The integrated assay values were floored at zero and normalized to the range of zero to one
adata.X[adata.X < 0]=0
mmin = np.amin(adata.X)
nor = (np.amax(adata.X)-mmin)
adata.X = (adata.X-mmin)/nor

In [7]:
## Prepare labels
time_bin = adata.obs['consensus.time'].copy()
celltype = adata.obs['celltype.anno']
celltype = celltype.replace('Quiescent Center', -1)
celltype = celltype.replace('Columella', 0)
celltype = celltype.replace('Lateral Root Cap', 1)
celltype = celltype.replace('Phloem', 2)
celltype = celltype.replace('Xylem', 3)
celltype = celltype.replace('Procambium', 4)
celltype = celltype.replace('Pericycle', 5)
celltype = celltype.replace('Endodermis', 6)
celltype = celltype.replace('Cortex', 7)
celltype = celltype.replace('Atrichoblast', 8)
celltype = celltype.replace('Trichoblast', 9)
for i in range(10):
    time_bin[np.where(celltype == i)[0]]=pd.qcut(pd.Series(adata.obs['consensus.time'][np.where(celltype == i)[0]]),q=10,labels=range(1,11)).to_numpy()
    
time_bin[np.where(celltype == -1)[0]] = 0 
labels = adata.obs['celltype.anno'].astype(str) + "_" + time_bin.astype(str)
labels = celltype.astype(str) + "_" + time_bin.astype(str)

## Prepare training, validation and test set
X_train, X_test, y_train, y_test = train_test_split(
            np.array(adata.X), labels, test_size=0.2, stratify=labels, shuffle=True, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(
            X_train, y_train, test_size=0.2, stratify=y_train, shuffle=True, random_state=0)
data = {}
data['X_train'] = X_train
data['y_train'] = y_train
data['X_val'] = X_val
data['y_val'] = y_val
data['X_test'] = X_test
data['y_test'] = y_test

## Save the input data for GeneSys

#with open("./genesys_root_data.pkl", 'wb') as file_handle:
#    pickle.dump(data, file_handle)

#### Load unseen scRNA-seq data

In [8]:
## Load unseen WT scRNA-seq data (The .h5ad file is converted from Seurat Objects from GEO GSE152766)
wt = sc.read_h5ad("/hpc/group/pbenfeylab/CheWei/scRNA-seq/Integrated_Objects/rswt_integrated.h5ad")

/hpc/group/pbenfeylab/ch416/miniconda3/envs/pytorch-gpu/lib/python3.8/site-packages/anndata/compat/__init__.py:232: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  warn(


In [9]:
wt

AnnData object with n_obs × n_vars = 22249 × 19269
    obs: 'orig.ident', 'time.celltype.anno.Li.crude', 'time.celltype.anno.Li', 'time.celltype.anno', 'time.celltype.anno.crude', 'celltype.anno.crude', 'celltype.anno', 'time.anno', 'consensus.time.group', 'CytoTRACE.time'
    var: 'SCT_features', '_index', 'features'
    uns: 'neighbors'
    obsm: 'X_pca', 'X_umap', 'X_umap_2D', 'X_umap_3D'
    varm: 'PCs'
    layers: 'SCT'
    obsp: 'distances'

In [10]:
##The integrated assay values were floored at zero and normalized to the range of zero to one
wt.X[wt.X < 0]=0
mmin = np.amin(wt.X)
nor = (np.amax(wt.X)-mmin)
wt.X = (wt.X-mmin)/nor

In [11]:
## Resize the number of features/genes to match GeneSys's input dimension for features/genes
mi = match(np.array(gene_list['features']).tolist(),wt.var.features.tolist())
mi = np.array(mi)

wtx = pd.DataFrame(wt.X.copy())
wtx[len(wtx.columns)] = pd.Series(0.0, index=np.arange(len(wtx)))
mi[np.where(mi==None)[0]] = len(wtx.columns)-1
wtx = wtx[pd.Series(mi)]
wtx.columns = np.array(gene_list['features']).tolist()

In [12]:
wtx.shape

(22249, 17513)

In [13]:
wtx

,AT1G05260,AT3G59370,AT2G36100,AT1G12080,AT1G12090,AT4G11290,AT5G42180,AT5G66390,AT2G32300,AT2G02130,...,AT4G06395,AT3G55440,AT3G03100,AT5G54760,AT2G33040,AT2G42680,AT5G11770,AT5G08290,AT5G53300,AT5G64400
0,0.0,1.0,0.000000,0.0,0.000000,0.0,0.280373,0.000000,0.0,0.629164,...,0.000000,0.026394,0.010529,0.000000,0.076983,0.188773,0.000000,0.000000,0.062446,0.023031
1,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.004356,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.074705,0.043984
2,0.0,0.0,0.000000,0.0,1.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.003400,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.657121,...,0.000000,0.000000,0.000000,0.011573,0.008235,0.253549,0.000000,0.020572,0.000000,0.015544
4,0.0,0.0,0.066269,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.119030,0.041525,0.069649,0.047870,0.102450,0.040937,0.089413,0.036108,0.066530
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22244,0.0,0.0,0.000000,0.0,1.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.007885,0.018090,0.046862,0.016902,0.012276,0.000000,0.000000,0.031478,0.018788,0.000000
22245,0.0,1.0,0.000000,1.0,0.259549,0.0,0.000000,0.155904,0.0,1.000000,...,0.000000,0.099126,0.172438,0.000000,0.047243,0.000000,0.000000,0.094165,0.096834,0.062185
22246,0.0,0.0,0.095308,0.0,0.770502,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.113197,0.000000,0.056963,0.000000,0.000000,0.000000,0.000000
22247,0.0,0.0,0.041253,0.0,0.000000,0.0,0.124179,0.127684,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.003071,0.000000,0.027698,0.003041,0.079789


In [14]:
## Prepare labels
time_bin = wt.obs['CytoTRACE.time'].copy()
celltype = wt.obs['celltype.anno.crude']
celltype = celltype.replace('Quiescent Center', -1)
celltype = celltype.replace('Columella', 0)
celltype = celltype.replace('Lateral Root Cap', 1)
celltype = celltype.replace('Phloem', 2)
celltype = celltype.replace('Xylem', 3)
celltype = celltype.replace('Procambium', 4)
celltype = celltype.replace('Pericycle', 5)
celltype = celltype.replace('Endodermis', 6)
celltype = celltype.replace('Cortex', 7)
celltype = celltype.replace('Atrichoblast', 8)
celltype = celltype.replace('Trichoblast', 9)
for i in range(10):
    time_bin[np.where(celltype == i)[0]]=pd.qcut(pd.Series(wt.obs['CytoTRACE.time'][np.where(celltype == i)[0]]),q=10,labels=range(1,11)).to_numpy()
    
time_bin[np.where(celltype == -1)[0]] = 0 
labels = wt.obs['celltype.anno.crude'].astype(str) + "_" + time_bin.astype(str)
labels = celltype.astype(str) + "_" + time_bin.astype(str)

## Prepare training, validation and test set
X_train, X_test, y_train, y_test = train_test_split(
            np.array(wtx), labels, test_size=0.2, stratify=labels, shuffle=True, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(
            X_train, y_train, test_size=0.2, stratify=y_train, shuffle=True, random_state=0)
data = {}
data['X_train'] = X_train
data['y_train'] = y_train
data['X_val'] = X_val
data['y_val'] = y_val
data['X_test'] = X_test
data['y_test'] = y_test

## Save the input data for GeneSys

#with open("./genesys_rswt_data.pkl", 'wb') as file_handle:
#    pickle.dump(data, file_handle)

#### Load mutant scRNA-seq data (shr and scr)

In [15]:
## Load mutant scRNA-seq data (The .h5ad file is converted from Seurat Objects from GEO GSE152766)
shr = sc.read_h5ad("/hpc/group/pbenfeylab/CheWei/scRNA-seq/Integrated_Objects/shr_integrated.h5ad")
scr = sc.read_h5ad("/hpc/group/pbenfeylab/CheWei/scRNA-seq/Integrated_Objects/scr_integrated.h5ad")

/hpc/group/pbenfeylab/ch416/miniconda3/envs/pytorch-gpu/lib/python3.8/site-packages/anndata/compat/__init__.py:232: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  warn(
/hpc/group/pbenfeylab/ch416/miniconda3/envs/pytorch-gpu/lib/python3.8/site-packages/anndata/compat/__init__.py:232: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  warn(


In [16]:
shr

AnnData object with n_obs × n_vars = 14612 × 19035
    obs: 'orig.ident', 'time.celltype.anno.Li.crude', 'time.celltype.anno.Li', 'time.celltype.anno', 'time.celltype.anno.crude', 'celltype.anno.crude', 'celltype.anno', 'time.anno', 'consensus.time.group', 'CytoTRACE.time'
    var: 'SCT_features', '_index', 'features'
    uns: 'neighbors'
    obsm: 'X_pca', 'X_umap', 'X_umap_2D', 'X_umap_3D', 'X_umap_50'
    varm: 'PCs'
    layers: 'SCT'
    obsp: 'distances'

In [17]:
scr

AnnData object with n_obs × n_vars = 13742 × 19368
    obs: 'orig.ident', 'time.celltype.anno.Li.crude', 'time.celltype.anno.Li', 'time.celltype.anno', 'time.celltype.anno.crude', 'celltype.anno.crude', 'celltype.anno', 'time.anno', 'consensus.time.group', 'CytoTRACE.time'
    var: 'SCT_features', '_index', 'features'
    uns: 'neighbors'
    obsm: 'X_pca', 'X_umap', 'X_umap_2D', 'X_umap_3D', 'X_umap_50'
    varm: 'PCs'
    layers: 'SCT'
    obsp: 'distances'

In [18]:
# Floored at 0 (And make sure ceiling at 10) and normalize to range 0-1
shr.X[shr.X < 0]=0
mmin = np.amin(shr.X)
nor = (np.amax(shr.X)-mmin)
shr.X = (shr.X-mmin)/nor
############################
scr.X[scr.X < 0]=0
mmin = np.amin(scr.X)
nor = (np.amax(scr.X)-mmin)
scr.X = (scr.X-mmin)/nor

In [22]:
## Resize the number of features/genes to match GeneSys's input dimension for features/genes
mi = match(np.array(gene_list['features']).tolist(),shr.var.features.tolist())
mi = np.array(mi)
shrx = pd.DataFrame(shr.X.copy())
shrx[len(shrx.columns)] = pd.Series(0.0, index=np.arange(len(shrx)))
mi[np.where(mi==None)[0]] = len(shrx.columns)-1
shrx = shrx[pd.Series(mi)]
shrx.columns = np.array(gene_list['features']).tolist()

In [23]:
shrx.shape

(14612, 17513)

In [24]:
## Prepare labels
time_bin = shr.obs['CytoTRACE.time'].copy()
celltype = shr.obs['celltype.anno.crude']
celltype = celltype.replace('Quiescent Center', -1)
celltype = celltype.replace('Columella', 0)
celltype = celltype.replace('Lateral Root Cap', 1)
celltype = celltype.replace('Phloem', 2)
celltype = celltype.replace('Xylem', 3)
celltype = celltype.replace('Procambium', 4)
celltype = celltype.replace('Pericycle', 5)
celltype = celltype.replace('Endodermis', 6)
celltype = celltype.replace('Cortex', 7)
celltype = celltype.replace('Atrichoblast', 8)
celltype = celltype.replace('Trichoblast', 9)
for i in range(10):
    time_bin[np.where(celltype == i)[0]]=pd.qcut(pd.Series(shr.obs['CytoTRACE.time'][np.where(celltype == i)[0]]),q=10,labels=range(1,11)).to_numpy()
    
time_bin[np.where(celltype == -1)[0]] = 0 
labels = shr.obs['celltype.anno.crude'].astype(str) + "_" + time_bin.astype(str)
labels = celltype.astype(str) + "_" + time_bin.astype(str)

## Prepare training, validation and test set
X_train, X_test, y_train, y_test = train_test_split(
            np.array(shrx), labels, test_size=0.2, stratify=labels, shuffle=True, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(
            X_train, y_train, test_size=0.2, stratify=y_train, shuffle=True, random_state=0)
data = {}
data['X_train'] = X_train
data['y_train'] = y_train
data['X_val'] = X_val
data['y_val'] = y_val
data['X_test'] = X_test
data['y_test'] = y_test

#with open("./genesys_shr_data.pkl", 'wb') as file_handle:
#    pickle.dump(data, file_handle)

In [25]:
## Resize the number of features/genes to match GeneSys's input dimension for features/genes
mi = match(np.array(gene_list['features']).tolist(),scr.var.features.tolist())
mi = np.array(mi)

scrx = pd.DataFrame(scr.X.copy())
scrx[len(scrx.columns)] = pd.Series(0.0, index=np.arange(len(scrx)))
mi[np.where(mi==None)[0]] = len(scrx.columns)-1
scrx = scrx[pd.Series(mi)]
scrx.columns = np.array(gene_list['features']).tolist()

In [26]:
scrx.shape

(13742, 17513)

In [27]:
## Prepare labels
time_bin = scr.obs['CytoTRACE.time'].copy()
celltype = scr.obs['celltype.anno.crude']
#celltype = celltype.replace('Quiescent Center', -1)
celltype = celltype.replace('Quiescent Center', 0)
celltype = celltype.replace('Columella', 0)
celltype = celltype.replace('Lateral Root Cap', 1)
celltype = celltype.replace('Phloem', 2)
celltype = celltype.replace('Xylem', 3)
celltype = celltype.replace('Procambium', 4)
celltype = celltype.replace('Pericycle', 5)
celltype = celltype.replace('Endodermis', 6)
celltype = celltype.replace('Cortex', 7)
celltype = celltype.replace('Atrichoblast', 8)
celltype = celltype.replace('Trichoblast', 9)
for i in range(10):
    time_bin[np.where(celltype == i)[0]]=pd.qcut(pd.Series(scr.obs['CytoTRACE.time'][np.where(celltype == i)[0]]),q=10,labels=range(1,11)).to_numpy()
    
time_bin[np.where(celltype == -1)[0]] = 0 
labels = shr.obs['celltype.anno.crude'].astype(str) + "_" + time_bin.astype(str)
labels = celltype.astype(str) + "_" + time_bin.astype(str)

## Prepare training, validation and test set
X_train, X_test, y_train, y_test = train_test_split(
            np.array(scrx), labels, test_size=0.2, stratify=labels, shuffle=True, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(
            X_train, y_train, test_size=0.2, stratify=y_train, shuffle=True, random_state=0)
data = {}
data['X_train'] = X_train
data['y_train'] = y_train
data['X_val'] = X_val
data['y_val'] = y_val
data['X_test'] = X_test
data['y_test'] = y_test

#with open("./genesys_scr_data.pkl", 'wb') as file_handle:
#    pickle.dump(data, file_handle)